In [1]:
import pandas as pd

In [113]:
ventas = pd.DataFrame({
    'persona': ['A', 'A', 'B', 'B', 'B', 'C', 'C', 'A'],
    'categoria': ['ropa', 'calzado', 'ropa', 'accesorios', 'calzado', 'ropa', 'accesorios', 'ropa'],
    'compra': [100, 200, 50, 75, 125, 300, 400, 50],
})
ventas.insert(3, 'times', 1)
ventas.head(10)

,persona,categoria,compra,times
0,A,ropa,100,1
1,A,calzado,200,1
2,B,ropa,50,1
3,B,accesorios,75,1
4,B,calzado,125,1
5,C,ropa,300,1
6,C,accesorios,400,1
7,A,ropa,50,1


In [114]:
total = ventas.groupby('persona').sum().reset_index()
total.rename(columns={'compra':'compra_total', 'times':'times_total'}, inplace=True)

ventas_por_persona_categoria = ventas.groupby(['persona', 'categoria'])
total_por_persona_categoria = ventas_por_persona_categoria[['compra','times']].sum().reset_index()
total_por_persona_categoria = total_por_persona_categoria.sort_values(by=['persona', 'categoria'], ascending=True)
total_general = pd.merge(total_por_persona_categoria, total, how='left', on=['persona'])   

total_general['proporcion_categoria'] = round(total_general['times'] / total_general['times_total'],3)
total_general['proporcion_monetaria'] = round(total_general['compra'] / total_general['compra_total'],3)

total_general.head()

C:\Users\Oscar\AppData\Local\Temp\ipykernel_232\930382070.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total = ventas.groupby('persona').sum().reset_index()


,persona,categoria,compra,times,compra_total,times_total,proporcion_categoria,proporcion_monetaria
0,A,calzado,200,1,350,3,0.333,0.571
1,A,ropa,150,2,350,3,0.667,0.429
2,B,accesorios,75,1,250,3,0.333,0.300
3,B,calzado,125,1,250,3,0.333,0.500
4,B,ropa,50,1,250,3,0.333,0.200


In [115]:
total.head()

,persona,compra_total,times_total
0,A,350,3
1,B,250,3
2,C,700,2


In [131]:
total_general_proporciones = total_general.loc[:,['persona','categoria','proporcion_categoria']] #,'proporcion_monetaria'
total_general_proporciones = total_general_proporciones.pivot_table(
    index='persona',
    columns = 'categoria',
    values= 'proporcion_categoria',
    )
total_general_proporciones = total_general_proporciones.add_prefix('cat_')
total_general_proporciones = total_general_proporciones.reset_index()
total_general_proporciones = total_general_proporciones.fillna(0)
total_general_proporciones

categoria,persona,cat_accesorios,cat_calzado,cat_ropa
0,A,0.000,0.333,0.667
1,B,0.333,0.333,0.333
2,C,0.500,0.000,0.500


In [139]:
total_general_monetario = total_general.loc[:,['persona','categoria','proporcion_monetaria']]
total_general_monetario = total_general_monetario.pivot_table(
    index='persona',
    columns = 'categoria',
    values= 'proporcion_monetaria',
    )
total_general_monetario = total_general_monetario.add_prefix('mon_')
total_general_monetario = total_general_monetario.reset_index()
total_general_monetario = total_general_monetario.fillna(0)
total_general_monetario

categoria,persona,mon_accesorios,mon_calzado,mon_ropa
0,A,0.000,0.571,0.429
1,B,0.300,0.500,0.200
2,C,0.571,0.000,0.429


In [140]:
# unirlos
df = total.merge(total_general_proporciones, on = ['persona'])
df = df.merge(total_general_monetario, on = ['persona'])

In [141]:
df.head(10)

,persona,compra_total,times_total,cat_accesorios,cat_calzado,cat_ropa,mon_accesorios,mon_calzado,mon_ropa
0,A,350,3,0.000,0.333,0.667,0.000,0.571,0.429
1,B,250,3,0.333,0.333,0.333,0.300,0.500,0.200
2,C,700,2,0.500,0.000,0.500,0.571,0.000,0.429
